In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import json
import numpy as np

# C.A.R.S. Chord

The following code creates the matrix I used for [this](http://bl.ocks.org/easadler/f36fcd4ac676eec59bd5) visualization. It shows how cars were exchanged in the C.A.R.S. program by the automaker's nationality for the 7 countries with the largest presence.
#### Steps
1. Scrape wikipedia to get auto-makers by country 
2. Transform data to get matrix of sums: clunker country -> new vehicle country 
3. Format into matrix specified by D3 

#### Technologies
* scraping: (requests, BeautifulSoup, regex)
* data manipulation: (Pandas, Numpy)

## 1. Scrape for automaker by country
url: https://en.wikipedia.org/wiki/List_of_automobile_manufacturers

In [15]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_automobile_manufacturers')

In [16]:
soup = BeautifulSoup(r.content, 'lxml')

In [17]:
# Parse html & create dictionary i.e. {automaker: Country}
auto_maker_dict = dict()
for headline in soup.findAll('h3'):
    if re.search(r'\[edit\]$', headline.text.strip()):
        country = headline.text[0:-6]
        if headline.findNextSibling().text.strip() == 'Current[edit]':
            for status in headline.findNextSiblings('ul', limit=2):
                for maker in status.find_all('a'):
                    auto_maker_dict[maker.text] = country               
        else:
            for maker in headline.find_next('ul').find_all('a'):
                auto_maker_dict[maker.text] = country               


In [18]:
# Change automakers to parent company
manual_maker = {'Oldsmobile': 'Ford', 'Mercury': 'Ford', 'Merkur': 'Ford', 'Pontiac': 'General Motors', 'Plymouth': 'General Motors'}
manual_country = {'General Motors': 'United States'}

In [19]:
# Add missing automakers
for key, val in manual_country.iteritems():
    auto_maker_dict[key] = val

In [20]:
# Create dataframe of automakers
df_am = pd.DataFrame({'maker': auto_maker_dict.keys(),'country': auto_maker_dict.values()})

## 2. Transform Data

In [39]:
df = pd.read_csv('data.txt')

In [22]:
df.columns

Index([u'vendor_id', u'dealer_name', u'address_line1', u'address_line2',
       u'address_line3', u'address_line4', u'city', u'state', u'ZIP',
       u'area_code', u'phone', u'invoice_id', u'invoice_num', u'invoice_date',
       u'sale_date', u'disposal_status', u'disposal_facility_nmvtis_id',
       u'disposal_facility_contact_info', u'sales_type', u'invoice_amount',
       u'trade_in_VIN', u'trade_in_vehicle_category', u'trade_in_make',
       u'trade_in_model', u'trade_in_year', u'trade_in_vehicle_drive_train',
       u'trade_in_mileage', u'trade_in_title_state',
       u'trade_in_registration_state', u'trade_in_registration_start',
       u'trade_in_registration_end', u'trade_in_insurance_start',
       u'trade_in_NMVTIS_flag', u'trade_in_odometer_reading',
       u'new_vehicle_VIN_trunc', u'new_vehicle_category', u'new_vehicle_make',
       u'new_vehicle_model', u'new_vehicle_year', u'new_vehicle_drive_train',
       u'new_vehicle_car_mileage', u'new_vehicle_MSRP'],
      dtype='o

In [40]:
# Change makers to parent type
for key, val in manual_maker.iteritems():
    df.ix[df['trade_in_make'] == key, 'trade_in_make'] = val
    df.ix[df['new_vehicle_make'] == key, 'new_vehicle_make'] = val

In [41]:
# Subset to important columns
df = df[['trade_in_make','new_vehicle_make']]

In [42]:
# Join automaker and main datasets twice to get country for both trade in and new vehicle
df_c = df.merge(df_am, left_on = 'trade_in_make', right_on='maker').merge(df_am, left_on = 'new_vehicle_make', right_on='maker')

In [43]:
df_c.columns

Index([u'trade_in_make', u'new_vehicle_make', u'country_x', u'maker_x',
       u'country_y', u'maker_y'],
      dtype='object')

In [44]:
# Subset to relevant columns
df_c = df_c[['country_x', 'country_y']]

In [45]:
# Rename columns
df_c.columns = ['trade_in','new_vehicle']

In [46]:
# Add count columns & groupby to get sums
df_c['count'] = 1
df_cg = df_c.groupby(['trade_in', 'new_vehicle']).sum().reset_index()

In [34]:
# Pivot table to get sums by countries
df_p = df_cg.pivot(index = 'trade_in', columns = 'new_vehicle', values = 'count')
df_p = df_p.fillna(0)

In [36]:
# Remove countries with too few counts to be seen on graph
df_p = df_p.ix[~df_p.index.isin(['France', 'Italy']),:]

In [ ]:
# Fill in countries with zero new vehicles
df_p['United Kingdom'] = 0

## 3. Prepare for d

In [37]:
# Sort index & columns
df_p = df_p.sort_index()
df_p = df_p.reindex_axis(sorted(df_p.columns), axis=1)

In [38]:
### Create big matrix
df_z = pd.DataFrame(data = np.zeros((len(df_p),len(df_p))), index = df_p.index)
dd = pd.concat([df_z, df_p.T], axis = 1)
dd.index = df_z.columns
dd1 = pd.concat([df_p, df_z], axis = 1)
dd1.columns = list(df_z.columns) + list(df_p.columns)
df_final = pd.concat([dd, dd1], axis=0)

In [226]:
# Create json file of countries and colors (format of chord example) dataframe: name colors
df_nc = pd.DataFrame(data = [(name, color) for name, color in zip(list(df_p.columns) + list(df_p.columns), range(len(df_temp)*2))], columns= ['name', 'color'])

df_nc.to_json('countries.json',orient='records')

In [227]:
# save final matrix
with open('matrix.json', 'w') as f:
    f.write(json.dumps(df_final.values.astype(int).tolist()))